# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [6]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [7]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 105451885,
 'calculationPrice': 'close',
 'change': -3.61,
 'changePercent': -0.0206,
 'close': 175.5,
 'closeSource': 'licafoif',
 'closeTime': 1669733968082,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 174.307,
 'delayedPriceTime': 1701668910598,
 'extendedChange': -0.41,
 'extendedChangePercent': -0.00249,
 'extendedPrice': 175.39,
 'extendedPriceTime': 1645047129377,
 'high': 179.79,
 'highSource': ' eiya5p dteedmelu ri1nc',
 'highTime': 1711678497529,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 169.36,
 'iexCloseTime': 1668806715381,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 176.2,
 'iexOpenTime': 1698388785374,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1720659609696,
 'latestPrice': 170.72,
 'latestSource': 'Close',
 'latestTime': 'February 11, 2022',
 'latestUpdate': 1648273550903,
 'latestVolume': 102156104,
 'low

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [11]:
price = data['latestPrice']
pe_ratio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [28]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [29]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index = True
        )
final_dataframe

        

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,140.70,17.47,N/A
1,AAL,18.06,-3.51,N/A
2,AAP,223.25,23.22,N/A
3,AAPL,172.20,28.82,N/A
4,ABBV,143.39,34.18,N/A
...,...,...,...,...
500,YUM,124.16,23.76,N/A
501,ZBH,117.78,30.36,N/A
502,ZBRA,452.31,14.3,N/A
503,ZION,73.91,7.19,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [30]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace= True)
final_dataframe.drop('index', axis = 1, inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,F,17.99,2.06,N/A
1,BIO,639.60,2.93,N/A
2,SYF,44.53,3.05,N/A
3,PHM,49.35,3.23,N/A
4,LEN,95.39,3.26,N/A
5,EBAY,59.77,3.32,N/A
6,COO,402.00,3.37,N/A
7,HPE,17.80,3.48,N/A
8,WHR,204.35,3.55,N/A
9,HPQ,37.49,3.6,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [31]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [37]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [40]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[row, 'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,F,17.99,2.06,1111
1,BIO,639.60,2.93,31
2,SYF,44.53,3.05,449
3,PHM,49.35,3.23,405
4,LEN,95.39,3.26,209
5,EBAY,59.77,3.32,334
6,COO,402.00,3.37,49
7,HPE,17.80,3.48,1123
8,WHR,204.35,3.55,97
9,HPQ,37.49,3.6,533


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [41]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [54]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN

        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
                ],
                index = rv_columns 
            ),
            ignore_index = True
        )
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.90,N/A,18.23,N/A,3.87,N/A,3.34,N/A,13.317731,N/A,6.494769,N/A,N/A
1,AAL,18.22,N/A,-3.57,N/A,-1.55,N/A,0.4709,N/A,-7.328022,N/A,1.477397,N/A,N/A
2,AAP,233.33,N/A,23.5,N/A,4.57,N/A,1.28,N/A,12.660711,N/A,2.919816,N/A,N/A
3,AAPL,169.02,N/A,29.13,N/A,39.55,N/A,7.31,N/A,22.326583,N/A,17.122990,N/A,N/A
4,ABBV,147.11,N/A,34.59,N/A,19.0,N/A,4.66,N/A,12.353321,N/A,8.746687,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.21,N/A,24.54,N/A,-4.77,N/A,5.66,N/A,20.671408,N/A,9.857863,N/A,N/A
501,ZBH,119.16,N/A,31.17,N/A,1.95,N/A,3.21,N/A,17.182102,N/A,5.487859,N/A,N/A
502,ZBRA,463.12,N/A,14.2,N/A,4.03,N/A,2.18,N/A,11.177320,N/A,4.732834,N/A,N/A
503,ZION,74.12,N/A,6.94,N/A,1.59,N/A,3.33,N/A,5.735515,N/A,3.110227,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [55]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.90,N/A,18.23,N/A,3.87,N/A,3.34,N/A,13.317731,N/A,6.494769,N/A,N/A
1,AAL,18.22,N/A,-3.57,N/A,-1.55,N/A,0.4709,N/A,-7.328022,N/A,1.477397,N/A,N/A
2,AAP,233.33,N/A,23.5,N/A,4.57,N/A,1.28,N/A,12.660711,N/A,2.919816,N/A,N/A
3,AAPL,169.02,N/A,29.13,N/A,39.55,N/A,7.31,N/A,22.326583,N/A,17.122990,N/A,N/A
4,ABBV,147.11,N/A,34.59,N/A,19.0,N/A,4.66,N/A,12.353321,N/A,8.746687,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.21,N/A,24.54,N/A,-4.77,N/A,5.66,N/A,20.671408,N/A,9.857863,N/A,N/A
501,ZBH,119.16,N/A,31.17,N/A,1.95,N/A,3.21,N/A,17.182102,N/A,5.487859,N/A,N/A
502,ZBRA,463.12,N/A,14.2,N/A,4.03,N/A,2.18,N/A,11.177320,N/A,4.732834,N/A,N/A
503,ZION,74.12,N/A,6.94,N/A,1.59,N/A,3.33,N/A,5.735515,N/A,3.110227,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [57]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.90,N/A,18.23,N/A,3.87,N/A,3.3400,N/A,13.317731,N/A,6.494769,N/A,N/A
1,AAL,18.22,N/A,-3.57,N/A,-1.55,N/A,0.4709,N/A,-7.328022,N/A,1.477397,N/A,N/A
2,AAP,233.33,N/A,23.50,N/A,4.57,N/A,1.2800,N/A,12.660711,N/A,2.919816,N/A,N/A
3,AAPL,169.02,N/A,29.13,N/A,39.55,N/A,7.3100,N/A,22.326583,N/A,17.122990,N/A,N/A
4,ABBV,147.11,N/A,34.59,N/A,19.00,N/A,4.6600,N/A,12.353321,N/A,8.746687,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.21,N/A,24.54,N/A,-4.77,N/A,5.6600,N/A,20.671408,N/A,9.857863,N/A,N/A
501,ZBH,119.16,N/A,31.17,N/A,1.95,N/A,3.2100,N/A,17.182102,N/A,5.487859,N/A,N/A
502,ZBRA,463.12,N/A,14.20,N/A,4.03,N/A,2.1800,N/A,11.177320,N/A,4.732834,N/A,N/A
503,ZION,74.12,N/A,6.94,N/A,1.59,N/A,3.3300,N/A,5.735515,N/A,3.110227,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [58]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [64]:
from scipy.stats import percentileofscore as score
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100


rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.90,N/A,18.23,0.413861,3.87,0.566337,3.3400,0.520792,13.317731,0.483168,6.494769,0.485149,N/A
1,AAL,18.22,N/A,-3.57,0.051485,-1.55,0.057426,0.4709,0.055446,-7.328022,0.021782,1.477397,0.063366,N/A
2,AAP,233.33,N/A,23.50,0.536634,4.57,0.677228,1.2800,0.213861,12.660711,0.459406,2.919816,0.164356,N/A
3,AAPL,169.02,N/A,29.13,0.667327,39.55,0.970297,7.3100,0.861386,22.326583,0.770297,17.122990,0.914851,N/A
4,ABBV,147.11,N/A,34.59,0.752475,19.00,0.932673,4.6600,0.711881,12.353321,0.441584,8.746687,0.677228,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.21,N/A,24.54,0.552475,-4.77,0.051485,5.6600,0.794059,20.671408,0.740594,9.857863,0.730693,N/A
501,ZBH,119.16,N/A,31.17,0.70495,1.95,0.29703,3.2100,0.506931,17.182102,0.631683,5.487859,0.39802,N/A
502,ZBRA,463.12,N/A,14.20,0.30396,4.03,0.584158,2.1800,0.382178,11.177320,0.380198,4.732834,0.330693,N/A
503,ZION,74.12,N/A,6.94,0.122772,1.59,0.214851,3.3300,0.518812,5.735515,0.10495,3.110227,0.178218,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [68]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,137.90,N/A,18.23,0.413861,3.87,0.566337,3.3400,0.520792,13.317731,0.483168,6.494769,0.485149,0.493861
1,AAL,18.22,N/A,-3.57,0.051485,-1.55,0.057426,0.4709,0.055446,-7.328022,0.021782,1.477397,0.063366,0.049901
2,AAP,233.33,N/A,23.50,0.536634,4.57,0.677228,1.2800,0.213861,12.660711,0.459406,2.919816,0.164356,0.410297
3,AAPL,169.02,N/A,29.13,0.667327,39.55,0.970297,7.3100,0.861386,22.326583,0.770297,17.122990,0.914851,0.836832
4,ABBV,147.11,N/A,34.59,0.752475,19.00,0.932673,4.6600,0.711881,12.353321,0.441584,8.746687,0.677228,0.703168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,126.21,N/A,24.54,0.552475,-4.77,0.051485,5.6600,0.794059,20.671408,0.740594,9.857863,0.730693,0.573861
501,ZBH,119.16,N/A,31.17,0.70495,1.95,0.29703,3.2100,0.506931,17.182102,0.631683,5.487859,0.39802,0.507723
502,ZBRA,463.12,N/A,14.20,0.30396,4.03,0.584158,2.1800,0.382178,11.177320,0.380198,4.732834,0.330693,0.396238
503,ZION,74.12,N/A,6.94,0.122772,1.59,0.214851,3.3300,0.518812,5.735515,0.10495,3.110227,0.178218,0.227921


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [72]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,F,17.93,N/A,2.03,0.061386,0.7281,0.079208,0.2628,0.013861,1.596262,0.027723,0.858590,0.029703,0.042376
1,GM,49.96,N/A,3.66,0.081188,0.6233,0.067327,0.2797,0.017822,1.021380,0.025743,0.900791,0.031683,0.044752
2,AAL,18.22,N/A,-3.57,0.051485,-1.5500,0.057426,0.4709,0.055446,-7.328022,0.021782,1.477397,0.063366,0.049901
3,HPQ,37.42,N/A,3.60,0.079208,-12.4000,0.041584,0.3259,0.019802,4.087315,0.057426,1.723898,0.085149,0.056634
4,UNM,30.58,N/A,7.62,0.136634,0.5468,0.063366,0.4518,0.049505,3.645533,0.045545,0.439496,0.005941,0.060198
5,SYF,45.66,N/A,3.11,0.065347,0.9252,0.10297,0.7124,0.091089,1.933810,0.029703,0.705359,0.019802,0.061782
6,L,64.72,N/A,5.22,0.09505,0.4471,0.059406,0.5366,0.063366,3.809100,0.051485,0.960930,0.039604,0.061782
7,PHM,48.85,N/A,3.38,0.073267,0.8166,0.093069,0.4402,0.045545,2.439457,0.033663,1.622739,0.073267,0.063762
8,PRU,123.51,N/A,6.56,0.106931,0.7411,0.081188,0.6731,0.079208,5.035114,0.079208,0.636516,0.013861,0.072079
9,WHR,204.12,N/A,3.53,0.077228,1.2300,0.138614,0.2730,0.015842,3.438347,0.043564,1.758571,0.091089,0.073267


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [75]:
portfolio_input()

In [76]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,F,17.93,11,2.03,0.061386,0.7281,0.079208,0.2628,0.013861,1.596262,0.027723,0.858590,0.029703,0.042376
1,GM,49.96,4,3.66,0.081188,0.6233,0.067327,0.2797,0.017822,1.021380,0.025743,0.900791,0.031683,0.044752
2,AAL,18.22,10,-3.57,0.051485,-1.5500,0.057426,0.4709,0.055446,-7.328022,0.021782,1.477397,0.063366,0.049901
3,HPQ,37.42,5,3.60,0.079208,-12.4000,0.041584,0.3259,0.019802,4.087315,0.057426,1.723898,0.085149,0.056634
4,UNM,30.58,6,7.62,0.136634,0.5468,0.063366,0.4518,0.049505,3.645533,0.045545,0.439496,0.005941,0.060198
5,SYF,45.66,4,3.11,0.065347,0.9252,0.10297,0.7124,0.091089,1.933810,0.029703,0.705359,0.019802,0.061782
6,L,64.72,3,5.22,0.09505,0.4471,0.059406,0.5366,0.063366,3.809100,0.051485,0.960930,0.039604,0.061782
7,PHM,48.85,4,3.38,0.073267,0.8166,0.093069,0.4402,0.045545,2.439457,0.033663,1.622739,0.073267,0.063762
8,PRU,123.51,1,6.56,0.106931,0.7411,0.081188,0.6731,0.079208,5.035114,0.079208,0.636516,0.013861,0.072079
9,WHR,204.12,0,3.53,0.077228,1.2300,0.138614,0.2730,0.015842,3.438347,0.043564,1.758571,0.091089,0.073267


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [78]:
writer = pd.ExcelWriter('value_strategy.xlsx',engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [79]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [80]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [81]:
writer.save()